In [1]:
import numpy as np
import os
import dill as pickle
import matplotlib.pyplot as plt
import pandas as pd
import sys
import hardware_control.wx_programs as wx
import hardware_control.bnc as bnc
from classes.generator import *
from hardware_control.hardware_config import *
from experiment_configuration.values import *
from classes.qubit_class import *
import daq.daq_programs_homo as daq
import classifiers.classifier as classifier
import seaborn as sns
import standard_sequences.rabi_tomo as tomo
import analysis.analysis as analysis
import time
import traceback

In [2]:
general_vals_dict["wx_offs"] = [0.7, 0, -0.08, 0]
q1 = Qubit(q1_dict, readout_dict)
q2 = Qubit(q2_dict, readout_dict)
readout = Readout(readout_dict)
print(f"{q1}\n{q2}")

wx_addr = wx.get_wx_address()
main_directory = r"C:\Users\quantum1\Documents"
save_dir = rf"{main_directory}\Python Scripts\Important Blue Fridge Python Files\New\nonlinear_QM\data"
target_bnc_address_6 = "USB0::0x03EB::0xAFFF::411-433500000-0753::INSTR"
bnc.set_bnc_output(
    general_vals_dict["qubit_bnc"], power_dBm=13, bnc_addr=target_bnc_address_6
)
bnc.set_bnc_output(
    readout_dict["RO_LO"],
    power_dBm=readout_dict["RO_LO_pwr"],
    bnc_addr=bnc_address["target_bnc_black"],
)
bnc.set_bnc_output(
    general_vals_dict["TWPA_freq"],
    general_vals_dict["TWPA_pwr"],
    bnc_addr=bnc_address["big_agilent"],
)

Qubit(ge_time=82, RO_LO_pwr=16, ro_dur=5000, qubit_thr=[-10000, -600], ef_ssm=-0.2568, ro_freq=6.72739, ef_time=49, ge_amp=0.795, RO_LO=6.6247, ef_amp=1.48, qubit_id=q1, RO_IF=None, ge_ssm=-0.1144, ro_amp=0.15, ROIF=0.10268999999999995, IQ_angle=110)
Qubit(ge_time=55, RO_LO_pwr=16, ro_dur=5000, qubit_thr=[-10000, 1900], ef_ssm=-0.2962, ro_freq=6.65554, ef_time=None, ge_amp=0.8, RO_LO=6.6247, ef_amp=1, qubit_id=q2, RO_IF=None, ge_ssm=-0.154, ro_amp=0.7, ROIF=0.030840000000000423, IQ_angle=25)


In [3]:
def run_rabi_tomo(
    q1: object,
    q2: object,
    general_vals_dict: dict,
    num_steps: int,
    sweep_time: float,
    swap_freq: float,
    swap_time: float,
    reps: int,
    tomography: str,
    J: float,
    y_ph: int,
    amp: float 
):
    """
    Runs a single instance of the nonhermitian ef Rabi experiment (with an e-swap to Q2)
    and processes the resulting IQ data to compute probabilities vs. time.

    Returns:
        df_prob (pd.DataFrame): A DataFrame whose index is the time (computed as
                                np.linspace(0, sweep_time/1000, num_steps)) and which has
                                columns ['P_f', 'P_e', 'P_g'] corresponding to the probabilities
                                of the f, e, and g states respectively.
    """
    # Run the experiment
    tomo.rabi_ef_swap_tomo(
        q1,
        q2,
        general_vals_dict,
        num_steps=num_steps,
        sweep_time=sweep_time,
        swap_freq=swap_freq,
        swap_time=swap_time,
        drive_amp_J=J,
        tomo_comp=tomography,
        y_ph=y_ph,
        amp=amp
    )

    wx.wx_set_and_amplitude_and_offset(
        amp=general_vals_dict["wx_amps"], offset=general_vals_dict["wx_offs"]
    )
    # Acquire the raw IQ data
    values = daq.run_daq_het_2q(
        q1, q2, num_patterns=num_steps, num_records_per_pattern=reps, verbose=False
    )

    # Retrieve raw IQ data from the acquired values
    I1_raw = values.rec_readout_1[0]
    Q1_raw = values.rec_readout_1[1]
    I2_raw = values.rec_readout_2[0]
    Q2_raw = values.rec_readout_2[1]

    # Build a DataFrame from the IQ data for classification
    IQ_df = pd.DataFrame({"I1": I1_raw, "Q1": Q1_raw, "I2": I2_raw, "Q2": Q2_raw})

    # Classify the IQ data (classifier.classify returns a DataFrame that has a 'predicted' column)
    classified = classifier.classify(IQ_df)
    states = classified["predicted"]
    # Reshape the predicted states.
    # (Assume that classifier.reshape_for_exp returns an array of shape (num_steps, reps),
    #  where each row corresponds to a time step and holds all the state measurements for that step.)
    states_reshaped = classifier.reshape_for_exp(states, reps, num_steps)
    probabilties = classifier.probabilities(states_reshaped)
    population = classifier.population(states_reshaped)

    # Compute time values as the index for the DataFrame.
    times = np.linspace(0, sweep_time / 1000, num_steps)

    # Build a DataFrame that holds the probability vs. time.
    df_prob = pd.DataFrame(
        {
            "time": times,
            "P_f": probabilties["P_f"],
            "P_e": probabilties["P_e"],
            "P_g": probabilties["P_g"],
        },
        index=times,
    )

    df_pop = pd.DataFrame(
        {
            "time": times,
            "Pop_f": population["Pop_f"],
            "Pop_e": population["Pop_e"],
            "Pop_g": population["Pop_g"],
        },
        index=times,
    )

    return df_prob, df_pop, values

In [4]:
# y_ph = 155
# reps = 1000
# sweep_time = 250
# swap_freq = -0.0194
# swap_time = 0.5 * 7 / abs(swap_freq)
# J = 10
# num_steps = 51
# sweep_start=0
# sweep_end=1.7
# sweep_steps=51
# sweep_vals = np.linspace(sweep_start, sweep_end, sweep_steps)

# # Define folders for plus and minus data (adjust folder names as needed)
# tomo_x_folder = os.path.join("amp_sweep", "x")
# tomo_y_folder = os.path.join("amp_sweep", "y")
# os.makedirs(tomo_x_folder, exist_ok=True)
# os.makedirs(tomo_y_folder, exist_ok=True)

# # Initialize dictionaries for storing the loaded or computed data
# return_x = {}   # for plus measurements
# return_y = {}   # for minus measurements

# # Optionally, if you want to keep track of the data in lists/arrays:
# sweep_data_x = {}
# sweep_data_y = {}

# # Loop over sweep values (e.g., different pi_phase values)
# for i, sweep_val in enumerate(sweep_vals):
#     # Build filenames using the sweep value as part of the name
#     x_filename = os.path.join(tomo_x_folder, f"amp={sweep_val:.5f}.pkl")
#     y_filename = os.path.join(tomo_y_folder, f"amp={sweep_val:.5f}.pkl")
    
#     # If both files exist, load them and store in the dictionaries
#     if os.path.exists(x_filename ) and os.path.exists(y_filename):
#         with open(x_filename , "rb") as f:
#             x_data = pickle.load(f)
#         with open(y_filename, "rb") as f:
#             y_data = pickle.load(f)
#         return_x[sweep_val] = x_data
#         return_x[sweep_val] = y_data
#         continue  # Skip to the next sweep value if files exist

#     # Otherwise, perform the measurement:
#     amp = sweep_val  # Set the current amplitude for the measurement       
#     tomography_x = "x"
#     tomography_y = "y"
#     # Perform the measurements for tomography "x" and "y"
#     df_prob_x, df_pop_x, values_x = run_rabi_tomo(
#         q1=q1,
#         q2=q2,
#         general_vals_dict=general_vals_dict,
#         num_steps=num_steps,
#         sweep_time=sweep_time,
#         swap_freq=swap_freq,
#         swap_time=swap_time,
#         reps=reps,
#         tomography=tomography_x,
#         J=J,
#         y_ph=y_ph,
#         amp=amp
#     )
#     df_prob_y, df_pop_y, values_y = run_rabi_tomo(
#         q1=q1,
#         q2=q2,
#         general_vals_dict=general_vals_dict,
#         num_steps=num_steps,
#         sweep_time=sweep_time,
#         swap_freq=swap_freq,
#         swap_time=swap_time,
#         reps=reps,
#         tomography=tomography_y,
#         J=J,
#         y_ph=y_ph,
#         amp=amp
#     )
    
#     # Save the measurement results to their respective pickle files
#     with open(x_filename, "wb") as f:
#         pickle.dump(df_prob_x, f)
#     with open(y_filename, "wb") as f:
#         pickle.dump(df_prob_y, f)
    
#     # Populate the dictionaries with the results, keyed by the amplitude value
#     return_x[sweep_val] = df_prob_x
#     return_y[sweep_val] = df_prob_y

# print("Measurement complete!")




In [5]:
# # Folders where the pickle files are stored
# tomo_x_folder = os.path.join("amp_sweep", "x")
# tomo_y_folder = os.path.join("amp_sweep", "y")

# # Function to extract the amplitude (sweep value) from the filename
# def get_amp_from_filename(filename):
#     # Assumes filename format "amp={value}.pkl"
#     base = os.path.basename(filename)
#     base_no_ext = os.path.splitext(base)[0]  # e.g., "amp=1.23456"
#     try:
#         amp_str = base_no_ext.split('=')[1]
#         return float(amp_str)
#     except (IndexError, ValueError):
#         return None

# # Get sorted list of pickle files based on the amplitude value
# x_files = sorted(
#     [os.path.join(tomo_x_folder, f) for f in os.listdir(tomo_x_folder) if f.endswith('.pkl')],
#     key=get_amp_from_filename
# )
# y_files = sorted(
#     [os.path.join(tomo_y_folder, f) for f in os.listdir(tomo_y_folder) if f.endswith('.pkl')],
#     key=get_amp_from_filename
# )

# # Lists to store amplitude values and the corresponding normalized P_f arrays for x
# amps_x = []
# norm_Pf_x_list = []
# time_array = None  # Will be taken from the first file

# for file in x_files:
#     amp = get_amp_from_filename(file)
#     if amp is None:
#         continue
#     amps_x.append(amp)
#     with open(file, "rb") as f:
#         # Each pickle file is assumed to contain a DataFrame
#         df = pickle.load(f)
#     # Normalize P_f as: P_f_norm = P_f / (P_f + P_e)
#     # (Assuming your DataFrame has columns "P_f" and "P_e" and a column "time")
#     df["normalized_P_f"] = df["P_f"] / (df["P_f"] + df["P_e"])
#     # Save the normalized data (as a numpy array) for each sweep value (row)
#     norm_Pf_x_list.append(df["normalized_P_f"].values)
#     if time_array is None:
#         # Use the 'time' column if available; otherwise, use the index
#         if "time" in df.columns:
#             time_array = df["time"].values
#         else:
#             time_array = df.index.values

# # Convert to numpy array (shape: [n_amp, n_time])
# norm_Pf_x = np.array(norm_Pf_x_list)
# amps_x = np.array(amps_x)

# # Repeat the same for the y data
# amps_y = []
# norm_Pf_y_list = []

# for file in y_files:
#     amp = get_amp_from_filename(file)
#     if amp is None:
#         continue
#     amps_y.append(amp)
#     with open(file, "rb") as f:
#         df = pickle.load(f)
#     df["normalized_P_f"] = df["P_f"] / (df["P_f"] + df["P_e"])
#     norm_Pf_y_list.append(df["normalized_P_f"].values)

# norm_Pf_y = np.array(norm_Pf_y_list)
# amps_y = np.array(amps_y)

# # Create a 2D plot for df_prob_x
# plt.figure(figsize=(8, 6))
# # pcolormesh expects X and Y coordinates. Here, time_array is along x and amplitude along y.
# plt.pcolormesh(time_array, amps_x, norm_Pf_x, shading='auto')
# plt.colorbar(label='Normalized P_f')
# plt.xlabel('Time')
# plt.ylabel('Amp')
# plt.title('2D Plot of Normalized $P_f$ (tomo_x)')
# plt.show()

# # Create a 2D plot for df_prob_y
# plt.figure(figsize=(8, 6))
# plt.pcolormesh(time_array, amps_y, norm_Pf_y, shading='auto')
# plt.colorbar(label='Normalized P_f')
# plt.xlabel('Time')
# plt.ylabel('Amp')
# plt.title('2D Plot of Normalized $P_f$ (tomo_y)')
# plt.show()

In [6]:

# # Folder where the tomo_y pickle files are stored
# tomo_y_folder = os.path.join("amp_sweep", "y")

# # Function to extract the amplitude (sweep value) from the filename.
# def get_amp_from_filename(filename):
#     # Assumes filename format "amp={value}.pkl"
#     base = os.path.basename(filename)
#     base_no_ext = os.path.splitext(base)[0]  # e.g., "amp=1.23456"
#     try:
#         amp_str = base_no_ext.split('=')[1]
#         return float(amp_str)
#     except (IndexError, ValueError):
#         return None

# # Get sorted list of pickle files for tomo_y data
# y_files = sorted(
#     [os.path.join(tomo_y_folder, f) for f in os.listdir(tomo_y_folder) if f.endswith('.pkl')],
#     key=get_amp_from_filename
# )

# amps_y = []
# std_deviation_y = []
# delta_Pf_time_series = []  # To store the time series of (normalized_P_f - p0)
# time_array = None  # Will be taken from the first file (assumed same for all)

# # Loop over each tomo_y file (each amplitude)
# for file in y_files:
#     amp = get_amp_from_filename(file)
#     if amp is None:
#         continue
#     amps_y.append(amp)
    
#     with open(file, "rb") as f:
#         df = pickle.load(f)
    
#     # Get the time axis (assumed identical for all files)
#     if time_array is None:
#         if "time" in df.columns:
#             time_array = df["time"].values
#         else:
#             time_array = df.index.values
    
#     # Compute normalized P_f: P_f_norm = P_f / (P_f + P_e)
#     df["normalized_P_f"] = df["P_f"] / (df["P_f"] + df["P_e"])
    
#     # Compute p0 as the average of max and min of normalized_P_f
#     p0 = (df["normalized_P_f"].max() + df["normalized_P_f"].min()) / 2.0
    
#     # Compute delta = normalized_P_f - p0
#     delta = df["normalized_P_f"] - p0
    
#     # Compute the standard deviation of delta over time
#     std_val = delta.std()
#     std_deviation_y.append(std_val)
    
#     # Save the delta time series for 2D plotting
#     delta_Pf_time_series.append(delta.values)

# # Convert lists to numpy arrays
# amps_y = np.array(amps_y)
# std_deviation_y = np.array(std_deviation_y)
# delta_Pf_array = np.array(delta_Pf_time_series)  # Shape: [n_amp, n_time]

# # Sort by amplitude for plotting consistency
# sort_idx = np.argsort(amps_y)
# amps_y_sorted = amps_y[sort_idx]
# std_deviation_y_sorted = std_deviation_y[sort_idx]
# delta_Pf_array_sorted = delta_Pf_array[sort_idx, :]



# # ------------------
# # Plot: Amp vs Standard Deviation of (Normalized_P_f - p0)
# plt.figure(figsize=(8,6))
# plt.plot(amps_y_sorted, std_deviation_y_sorted, marker='o', linestyle='-')
# plt.xlabel('Amp')
# plt.ylabel('Std of (Normalized $P_f$ - p0)')
# plt.title('Amp vs Standard Deviation of (Normalized $P_f$ - p0)')
# plt.grid(True)

# # Find the amplitude with the lowest standard deviation (lowest peak)
# min_idx = np.argmin(std_deviation_y_sorted)
# best_amp = amps_y_sorted[min_idx]
# plt.axvline(best_amp, color='red', linestyle='--', label=f'Lowest at amp={best_amp:.5f}')
# plt.legend()
# plt.show()

# print("Amplitude with lowest standard deviation:", best_amp)


In [7]:
# import os
# import pickle
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from scipy.signal import find_peaks

# # --- Process tomo_x data ---
# amps_x = []
# norm_Pf_x_list = []
# time_array = None  # Will be taken from the first file

# for file in x_files:
#     amp = get_amp_from_filename(file)
#     if amp is None:
#         continue
#     amps_x.append(amp)
#     with open(file, "rb") as f:
#         # Each pickle file is assumed to contain a DataFrame
#         df = pickle.load(f)
#     # Normalize P_f as: P_f_norm = P_f / (P_f + P_e)
#     df["normalized_P_f"] = df["P_f"] / (df["P_f"] + df["P_e"])
#     norm_Pf_x_list.append(df["normalized_P_f"].values)
#     if time_array is None:
#         # Use the 'time' column if available; otherwise, use the index
#         if "time" in df.columns:
#             time_array = df["time"].values
#         else:
#             time_array = df.index.values

# norm_Pf_x = np.array(norm_Pf_x_list)
# amps_x = np.array(amps_x)

# # --- Create 2D plot for tomo_x data ---
# plt.figure(figsize=(8, 6))
# plt.pcolormesh(time_array, amps_x, norm_Pf_x, shading='auto')
# plt.colorbar(label='Normalized P_f')
# plt.xlabel(r'Time ($\mu$s)')
# plt.ylabel('Amp')
# plt.title('2D Plot of Normalized $P_f$ (tomo_x)')

# # --- Find peaks in the row corresponding to amp = 0 ---
# idx_amp0 = np.argmin(np.abs(amps_x - 0))
# row_amp0 = norm_Pf_x[idx_amp0, :]

# # Use find_peaks to locate local maxima in this row
# peaks, properties = find_peaks(row_amp0)

# if len(peaks) == 0:
#     print("No peaks found in the row for amp = 0.")
# else:
#     for peak_idx in peaks:
#         t_val = time_array[peak_idx]
#         # Draw vertical line at the detected peak time
#         plt.axvline(x=t_val, color='white', linestyle='--', label=f"t_peak={t_val:.3f}")
        
#         # Extract the vertical profile at the peak time
#         vertical_profile = norm_Pf_x[:, peak_idx]
        
#         # Find where vertical_profile crosses 0.5 using linear interpolation
#         amp_at_05 = np.nan
#         for i in range(len(amps_x) - 1):
#             y1 = vertical_profile[i]
#             y2 = vertical_profile[i + 1]
#             # Check if 0.5 lies between y1 and y2
#             if (y1 - 0.5) * (y2 - 0.5) < 0:
#                 x1, x2 = amps_x[i], amps_x[i + 1]
#                 amp_at_05 = x1 + (0.5 - y1) * (x2 - x1) / (y2 - y1)
#                 break
                
#         if not np.isnan(amp_at_05):
#             plt.axhline(y=amp_at_05, color='white', linestyle=':', 
#                         label=f"Amp@0.5 at t={t_val:.3f}: {amp_at_05:.3f}")
#             print(f"At t = {t_val:.3f}, amplitude where normalized P_f = 0.5 is: {amp_at_05:.3f}")
#         else:
#             print(f"At t = {t_val:.3f}, no crossing of 0.5 was found in the vertical profile.")

# plt.legend()
# plt.show()



In [8]:
amp=1.428
reps = 1000
sweep_time = 250
swap_freq = -0.0194
swap_time = 0.5 * 7 / abs(swap_freq)
J = 10
num_steps = 51
sweep_start=0
sweep_end=360
sweep_steps=21
sweep_vals = np.linspace(sweep_start, sweep_end, sweep_steps)

# Define folders for plus and minus data (adjust folder names as needed)
tomo_x_folder = os.path.join("phase_sweep", "x")
tomo_y_folder = os.path.join("phase_sweep", "y")
os.makedirs(tomo_x_folder, exist_ok=True)
os.makedirs(tomo_y_folder, exist_ok=True)



def run_measurements():
    # Dictionaries to store the loaded or computed data
    return_x = {}   # For tomography "x" measurements
    return_y = {}   # For tomography "y" measurements


    # Loop over sweep values (e.g., different phase values)
    for i, sweep_val in enumerate(sweep_vals):
        # Build filenames using the sweep value as part of the name
        x_filename = os.path.join(tomo_x_folder, f"phase={sweep_val:.5f}.pkl")
        y_filename = os.path.join(tomo_y_folder, f"phase={sweep_val:.5f}.pkl")
        
        # If both files exist, load them and store in the dictionaries
        if os.path.exists(x_filename) and os.path.exists(y_filename):
            with open(x_filename, "rb") as f:
                x_data = pickle.load(f)
            with open(y_filename, "rb") as f:
                y_data = pickle.load(f)
            return_x[sweep_val] = x_data
            return_y[sweep_val] = y_data
            continue  # Skip to the next sweep value if files exist
        
        # Otherwise, perform the measurement:
        # In your case, y_ph is set by sweep_val
        y_ph = sweep_val
        tomography_x = "x"
        tomography_y = "y"
        
        # Perform the measurements for tomography "x" and "y"
        df_prob_x, df_pop_x, values_x = run_rabi_tomo(
            q1=q1,
            q2=q2,
            general_vals_dict=general_vals_dict,
            num_steps=num_steps,
            sweep_time=sweep_time,
            swap_freq=swap_freq,
            swap_time=swap_time,
            reps=reps,
            tomography=tomography_x,
            J=J,
            y_ph=y_ph,
            amp=amp
        )
        df_prob_y, df_pop_y, values_y = run_rabi_tomo(
            q1=q1,
            q2=q2,
            general_vals_dict=general_vals_dict,
            num_steps=num_steps,
            sweep_time=sweep_time,
            swap_freq=swap_freq,
            swap_time=swap_time,
            reps=reps,
            tomography=tomography_y,
            J=J,
            y_ph=y_ph,
            amp=amp
        )
        
        # Save the measurement results to their respective pickle files
        with open(x_filename, "wb") as f:
            pickle.dump(df_prob_x, f)
        with open(y_filename, "wb") as f:
            pickle.dump(df_prob_y, f)
        
        # Populate the dictionaries with the results, keyed by the sweep value
        return_x[sweep_val] = df_prob_x
        return_y[sweep_val] = df_prob_y

    print("Measurement complete!")
    return return_x, return_y

# Main loop: if an error occurs, print the error, wait 5 seconds, and try again
while True:
    try:
        rx, ry = run_measurements()
        break  # Exit loop when measurements complete successfully
    except Exception as e:
        print("An error occurred:")
        traceback.print_exc()
        print("Restarting in 5 seconds...")
        time.sleep(5)


writing to C:\arbsequences\strong_dispersive_withPython\test_pulse_ringupdown_bin\
writing ch1
writing ch2
writing ch3
writing ch4
loading C:\arbsequences\strong_dispersive_withPython\test_pulse_ringupdown_bin\
num_steps 51
loading ch1
loading ch2
loading ch3
loading ch4
0, No error
0, No error
0, No error
0, No error
0, No error
0, No error
Patterns: 51
Records per pattern: 1000
Buffers per acquistion: 17
DAQ samples per pattern: 8192
0, No error
Capturing 17 buffers. Press <enter> to abort
Capture completed in 6.999956 sec
Captured 17 buffers (2.428587 buffers per sec)
Captured 52224 records (7460.618184 records per sec)
Transferred 855638016 bytes (122234768.331331 bytes per sec)
writing to C:\arbsequences\strong_dispersive_withPython\test_pulse_ringupdown_bin\
writing ch1
writing ch2
writing ch3
writing ch4
loading C:\arbsequences\strong_dispersive_withPython\test_pulse_ringupdown_bin\
num_steps 51
loading ch1
loading ch2
loading ch3
loading ch4
0, No error
0, No error
0, No error
